In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os



In [2]:
in_file="raw_data_05-May'25.csv"
out_file="all_flow_data_04th May'25.csv"

In [3]:

raw_data=pd.read_csv(in_file,low_memory=False,memory_map=True)
data=pd.read_csv("format.csv",memory_map=True)
table_data=pd.read_csv("table_data.csv",memory_map=True)    



In [4]:
raw_data.head(3)

,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,sub_stage1
0,NaN,f77e507a-c5ff-4414-8b93-315133d7b1c5,1504448001,112,LEAD_CREATED,2025-05-05 12:23:49.000,2025-05-05 12:23:49.000,NaN,NaN,INDIVIDUAL,"[""Paytm QR""]",NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-05-05 12:23:49.000,NaN
1,1.372647e+09,158de65a-8378-427d-8cd4-12984d8b8658,1838032847,112,LEAD_CREATED,2025-05-01 14:06:10.000,2025-05-01 14:06:10.000,NaN,NaN,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-05-01 14:06:10.000,NaN
2,1.324878e+09,c33743bd-2ee7-45df-a0b5-c496d9c36005,1822347005,156,LEAD_SUCCESSFULLY_CLOSED,2025-05-05 11:35:23.000,2025-05-05 11:58:04.000,NaN,NaN,INDIVIDUAL,"[""Paytm QR""]","{""PANEL_SUCCESS"":1}",True,vLRXvb49620015692000,2025-05-05 11:51:16.000,General Trade QR_SB_EDC,2025-05-05 11:46:59.000,2025-05-05 11:51:03.000,2025-05-05 11:51:03.000,NaN


In [5]:

raw_data.insert(5,'q',0)
len(raw_data)

86303

In [6]:
table_map = table_data.set_index('NODE ID')['Lead Status'].to_dict()
raw_data['q'] = raw_data['workflow_node_id'].map(table_map)



In [7]:
raw_data.loc[raw_data['sub_stage1']=='QC_AUTO_APPROVED','q']='Leads QC Auto Approved'

In [8]:
raw_data['q'].value_counts()

q
Merchant Live                                                       48314
Lead Created/Posted (Pending at Sales team for  Document Upload)    28301
Documents Rejected (Pending at Sales Team)                           6032
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         782
Leads Rejected in Risk Review                                           3
Leads QC Auto Approved                                                  1
Name: count, dtype: int64

In [9]:
raw_data=raw_data.dropna(subset=['q'])

raw_data['q'].value_counts()

q
Merchant Live                                                       48314
Lead Created/Posted (Pending at Sales team for  Document Upload)    28301
Documents Rejected (Pending at Sales Team)                           6032
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         782
Leads Rejected in Risk Review                                           3
Leads QC Auto Approved                                                  1
Name: count, dtype: int64

In [10]:
raw_data['remove_date']=raw_data['updated_at'].copy()

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])

raw_data['remove_date'] = raw_data['remove_date'].dt.strftime('%Y-%m-%d')

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])
len(raw_data)


83433

In [11]:
#print(pd.Timestamp.now().normalize()-timedelta(days=1))

In [12]:

current_timestamp = pd.Timestamp.now().normalize()

raw_data=raw_data[raw_data['remove_date']!=current_timestamp]

In [13]:
raw_data=raw_data.drop(columns=['remove_date'])



In [14]:
raw_data['solution_type_level_3'].unique()

array([nan, 'ACCOUNT_UPGRADE_LENDING', 'KYC_UPDATE', 'PROFILE_UPDATE',
       'DIY', 'ACCOUNT_UPGRADE'], dtype=object)

In [15]:
raw_data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE                 2308
ACCOUNT_UPGRADE_LENDING    1244
PROFILE_UPDATE               25
ACCOUNT_UPGRADE               6
DIY                           2
Name: count, dtype: int64

In [16]:
raw_data=raw_data[raw_data['solution_type_level_3']!='YBL_LIMIT_ENHANCEMENT']

raw_data['solution_type_level_3'].unique()

array([nan, 'ACCOUNT_UPGRADE_LENDING', 'KYC_UPDATE', 'PROFILE_UPDATE',
       'DIY', 'ACCOUNT_UPGRADE'], dtype=object)

In [17]:
raw_data['solution_type_level_3']=raw_data['solution_type_level_3'].replace(['ACCOUNT_UPGRADE_LENDING','PROFILE_UPDATE'],'ACCOUNT_UPGRADE')

raw_data['solution_type_level_3'].unique()

array([nan, 'ACCOUNT_UPGRADE', 'KYC_UPDATE', 'DIY'], dtype=object)

In [18]:
raw_data.loc[raw_data['agent_team']=='3P FSE','solution_type_level_3']='3P FSE'

In [19]:
raw_data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE         2191
ACCOUNT_UPGRADE    1275
3P FSE              129
DIY                   2
Name: count, dtype: int64

In [20]:
print(len(raw_data))
raw_data.head(3)

64893


,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,q,created_at,updated_at,solution_type_level_2,solution_type_level_3,...,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,sub_stage1
1,1.372647e+09,158de65a-8378-427d-8cd4-12984d8b8658,1838032847,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-05-01 14:06:10.000,2025-05-01 14:06:10.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-05-01 14:06:10.000,NaN
4,1.491492e+09,40515a5e-f1e0-4507-968c-d4cf36b4537f,397374112,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-01 13:12:30.000,2025-05-01 13:37:42.000,NaN,NaN,...,NaN,"{""PANEL_SUCCESS"":1}",True,qucznn04531727751160,2025-05-01 13:37:10.000,General Trade QR_SB_EDC,2025-05-01 13:33:46.000,2025-05-01 13:36:55.000,2025-05-01 13:36:55.000,NaN
5,1.451714e+09,d643a08f-f30c-4fc9-a5cd-02fb795f2041,1313171549,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-03 16:03:48.000,2025-05-03 17:58:32.000,NaN,NaN,...,"[""Paytm QR""]","{""QC_REJECTED"":2,""PANEL_SUCCESS"":1}",True,CEGyew92963072307811,2025-05-03 17:57:54.000,General Trade QR_SB_EDC,2025-05-03 17:43:57.000,2025-05-03 17:57:37.000,2025-05-03 17:57:37.000,NaN


In [21]:
data[['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp',
       'Product_Context', 'Name_Match_Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp',
       'Stage movement count', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3']]=raw_data[['cust_id', 'lead_id','entity_type','workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at','products_context','name_match_status','final_timestamp','panelqc_max_created','qc_pending_max_created','sbstage_count','fse_cust_id','agent_team','mid','mid_creation_date','solution_type_level_3']]
       
len(data)

64893

In [22]:
data.head(3)


,Merchant Cust ID,Lead_ID,Entity_Type,Node id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,...,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3
1,1.372647e+09,158de65a-8378-427d-8cd4-12984d8b8658,INDIVIDUAL,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-05-01 14:06:10.000,2025-05-01 14:06:10.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1838032847,General Trade QR_SB_EDC,NaN,NaN,NaN
4,1.491492e+09,40515a5e-f1e0-4507-968c-d4cf36b4537f,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-01 13:12:30.000,2025-05-01 13:37:42.000,NaN,NaN,...,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,397374112,General Trade QR_SB_EDC,qucznn04531727751160,2025-05-01 13:37:10.000,NaN
5,1.451714e+09,d643a08f-f30c-4fc9-a5cd-02fb795f2041,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-03 16:03:48.000,2025-05-03 17:58:32.000,NaN,"[""Paytm QR""]",...,NaN,NaN,NaN,"{""QC_REJECTED"":2,""PANEL_SUCCESS"":1}",NaN,1313171549,General Trade QR_SB_EDC,CEGyew92963072307811,2025-05-03 17:57:54.000,NaN


In [23]:
data['Updated Timestamp']=pd.to_datetime(data['Updated Timestamp'])
data['Created Timestamp']=pd.to_datetime(data['Created Timestamp'])
data['QC Panel Successful TimeStamp']=pd.to_datetime(data['QC Panel Successful TimeStamp'])


In [24]:
#current_date = datetime.now() - timedelta(days=2)

#current_timestamp = pd.Timestamp.now().normalize()-timedelta(days=1)


#print(current_timestamp)

#print(current_date)

In [25]:
user_date = '2024-10-15'  # Example: YYYY-MM-DD format

# Convert the combined string into a datetime object
custom_timestamp = datetime.strptime(user_date, '%Y-%m-%d')

print(custom_timestamp)
print(type(custom_timestamp))

current_day = custom_timestamp.day
current_month = custom_timestamp.month
current_year = custom_timestamp.year

print(current_day)
print(current_month)
print(current_year)


2024-10-15 00:00:00
<class 'datetime.datetime'>
15
10
2024


In [26]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[(data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Last 7 days'

2025-04-28 17:02:55.118129


In [27]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[(data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Yesterday'

In [28]:
data['Ageing']=data['Ageing'].fillna('')

In [29]:
data['Ageing'].value_counts()

Ageing
             62567
Yesterday     2326
Name: count, dtype: int64

In [30]:


data.loc[data['Name_Match_Status']==True, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Matched','10k MID created']

data.loc[data['Name_Match_Status']==False, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Not Matched','1Rs MID created']

In [31]:
data['Name_Match_Status']=data['Name_Match_Status'].fillna('')

In [32]:
data['Name_Match_Status'].value_counts()

Name_Match_Status
Name Matched        40141
                    13801
Name Not Matched    10951
Name: count, dtype: int64

In [33]:
data['Product_Context'].value_counts()

Product_Context
["Paytm QR"]     34779
["EDC"]           3728
["TAP_N_PAY"]       98
Name: count, dtype: int64

In [34]:
data.loc[data['Product_Context'].isin(['[]','["EDC"]','["TAP_N_PAY"]']),'Product_Context']='EDC'
data['Product_Context'].value_counts()

Product_Context
["Paytm QR"]    34779
EDC              3826
Name: count, dtype: int64

In [35]:
data.loc[~data['Product_Context'].isin(['EDC']),'Product_Context']='NON_EDC'
data['Product_Context'].value_counts()

Product_Context
NON_EDC    61067
EDC         3826
Name: count, dtype: int64

In [36]:
data['mid_created_date']=pd.to_datetime(data['mid_created_date'],errors='coerce')

In [37]:
data['mid_created_date'].head(10)

1                    NaT
4    2025-05-01 13:37:10
5    2025-05-03 17:57:54
8    2020-05-23 15:31:06
10   2025-05-03 14:31:59
11   2025-05-01 11:35:57
12   2025-05-03 11:46:55
13   2025-05-01 17:17:46
15   2025-05-02 16:32:18
18                   NaT
Name: mid_created_date, dtype: datetime64[ns]

In [38]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [39]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day<current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='More Than 7 Days'

2025-04-28 17:02:55.372590


In [40]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='6-7 Days'

In [41]:
current_date = datetime.now() - timedelta(days=5)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='4-5 Days'

In [42]:

current_date = datetime.now() - timedelta(days=3)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='2-3 Days'

In [43]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='Less Than 1 Day'

In [44]:
data['Rejection Ageing'].value_counts()

Rejection Ageing
2-3 Days           3613
Less Than 1 Day     193
Name: count, dtype: int64

In [45]:
data['Rejection Ageing'].value_counts()

Rejection Ageing
2-3 Days           3613
Less Than 1 Day     193
Name: count, dtype: int64

In [46]:
data.loc[~data['Entity_Type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'Entity_Type']='NON_PROPRIETORSHIP'

In [47]:
data['Entity_Type'].value_counts()

Entity_Type
INDIVIDUAL            59462
PROPRIETORSHIP         5044
NON_PROPRIETORSHIP      387
Name: count, dtype: int64

In [48]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [49]:
data['Lead Status'].value_counts()

Lead Status
Merchant Live                                                       38088
Lead Created/Posted (Pending at Sales team for  Document Upload)    21545
Documents Rejected (Pending at Sales Team)                           4775
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         482
Leads Rejected in Risk Review                                           2
Leads QC Auto Approved                                                  1
Name: count, dtype: int64

In [50]:
null_rejected_timeStamp=data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]


null_rejected_timeStamp=null_rejected_timeStamp[['Lead_ID']]

print(len(null_rejected_timeStamp))

null_rejected_timeStamp['id']="'" + null_rejected_timeStamp['Lead_ID'] + "'" + ","

null_rejected_timeStamp.head(3)

null_rejected_timeStamp.to_csv("rejected_null.csv", index=False)


1


In [51]:
#dghdsg

In [52]:
# raise Exception("Stop execution here")

print(len(data))

64893


In [53]:
rejected_null_value=pd.read_csv('rejected_null_value.csv')

rejected_null_value['q']=pd.to_datetime(rejected_null_value['q'],format='%d-%m-%Y %H:%M:%S')

print(rejected_null_value.columns)

Index(['lead_id', 'q'], dtype='object')


In [54]:
print(rejected_null_value.head(3))
print(len(data))

                                lead_id                   q
0  d6143059-caa9-4353-93ff-cde985e787a5 2025-02-17 16:56:22
64893


In [55]:

# Aggregate the right DataFrame by taking the first occurrence per lead_id
#rejected_agg = rejected_null_value.groupby('lead_id').first().reset_index()

# Merge the DataFrames on the single column
#data = data.merge(rejected_agg[['lead_id', 'Status']], left_on='Lead_ID', right_on='lead_id', how='left')

# Fill in values based on conditions
#data.loc[(data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()), 'Last QC Pending Timestamp'] = data['q']


In [56]:
data=data.merge(rejected_null_value,left_on='Lead_ID',right_on='lead_id',how='left')

data.loc[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()) ,'Last QC Pending Timestamp']=data['q']

data=data.drop(columns=['lead_id','q'])

print(data.columns)
print(len(data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]))
print(len(data))

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')
0
64893


In [57]:
data['Final TimeStamp']=pd.to_datetime(data['Final TimeStamp'])
data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'])

In [58]:
# type(data['Final TimeStamp'][0])

print(type(data['Last QC Pending Timestamp'][0]))
print(len(data))

<class 'pandas._libs.tslibs.nattype.NaTType'>
64893


In [59]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [60]:
data['Last QC Pending Timestamp'].head(10)

0                   NaT
1   2025-05-01 13:33:46
2   2025-05-03 17:43:57
3                   NaT
4   2025-05-03 14:13:24
5   2025-05-01 11:31:27
6   2025-05-03 11:31:38
7   2025-05-01 17:14:00
8   2025-05-02 16:30:08
9                   NaT
Name: Last QC Pending Timestamp, dtype: datetime64[ns]

In [61]:


data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')

# Define the start and end times
start_time = time(8, 30, 0)
end_time = time(21, 30, 0)



# Apply the condition to classify as 'Intime' or 'Offtime' based on the time component of 'N'
data['Working TimeSlab'] = data['Last QC Pending Timestamp'].apply(lambda x: 'Offtime' if pd.isna(x) else ('Intime' if start_time <= x.time() <= end_time else 'Offtime'))


data['Working TimeSlab'].head(10)

0    Offtime
1     Intime
2     Intime
3    Offtime
4     Intime
5     Intime
6     Intime
7     Intime
8     Intime
9    Offtime
Name: Working TimeSlab, dtype: object

In [62]:
data['Working TimeSlab'].value_counts()

Working TimeSlab
Intime     42615
Offtime    22278
Name: count, dtype: int64

In [63]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [64]:
out_time_start = pd.to_datetime("21:30").time()
out_time_end = pd.to_datetime("23:59:59").time()
morning_time_end = pd.to_datetime("08:30").time()

data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')


# Apply conditions to create column R
def process_timestamp(row):
    timestamp = row['Last QC Pending Timestamp']
    
    # Check if timestamp is NaT
    if pd.isna(timestamp):
        return np.nan
    
    time = timestamp.time()
    
    if row['Working TimeSlab'] == 'Offtime':
        if out_time_start <= time <= out_time_end:
            # Condition 1: Increment date by 1
            return timestamp + pd.DateOffset(days=1)
        elif time < morning_time_end:
            # Condition 2: Return the same timestamp
            return timestamp
    # Condition 3: Return blank for 'intime'
    return np.nan


data['New Off Time'] = data.apply(process_timestamp, axis=1)

In [65]:
# Function to create S column based on R
def generate_s(row):
    if pd.notna(row['New Off Time']):  # If R is not NaT or blank
        # Set time to 08:30 on the same date as R
        return row['New Off Time'].replace(hour=8, minute=30, second=0)
    else:
        # If R is NaT or blank, copy the value from Q
        return row['Last QC Pending Timestamp']
    

data['New Last QC Pending Time'] = data.apply(generate_s, axis=1)

In [66]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [67]:
# Calculate the difference between L and S and format it as HH:mm:ss
def calculate_difference(row):
    if isinstance(row['New Last QC Pending Time'], pd.Timestamp) and isinstance(row['Final TimeStamp'], pd.Timestamp):
        diff = row['Final TimeStamp'] - row['New Last QC Pending Time']
        return diff  # Extract HH:mm:ss from Timedelta
    return np.nan  # Return NaN if either L or S is not a timestamp

# Apply function to create Difference column
data['TAT'] = data.apply(calculate_difference, axis=1)

In [68]:
data['TAT'].head(10)

0               NaT
1   0 days 00:03:09
2   0 days 00:13:40
3               NaT
4   0 days 00:18:16
5   0 days 00:04:16
6   0 days 00:15:03
7   0 days 00:03:37
8   0 days 00:01:58
9               NaT
Name: TAT, dtype: timedelta64[ns]

In [69]:
# Update TAT where it is negative to be the difference between L and N
def adjust_negative_tat(row):
    if pd.notna(row['TAT']) and row['TAT'] < pd.Timedelta(0):  # Check if TAT is negative
        if isinstance(row['Final TimeStamp'], pd.Timestamp) and isinstance(row['Last QC Pending Timestamp'], pd.Timestamp):
            return row['Final TimeStamp'] - row['Last QC Pending Timestamp']  # Calculate L - N if TAT is negative
    return row['TAT']  # Keep original TAT if not negative

# Apply function to adjust negative TAT values
data['TAT'] = data.apply(adjust_negative_tat, axis=1)

In [70]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

      Final TimeStamp Last QC Pending Timestamp             TAT
0 2025-05-01 14:06:10                       NaT             NaT
1 2025-05-01 13:36:55       2025-05-01 13:33:46 0 days 00:03:09
2 2025-05-03 17:57:37       2025-05-03 17:43:57 0 days 00:13:40
3 2025-05-01 12:08:31                       NaT             NaT
4 2025-05-03 14:31:40       2025-05-03 14:13:24 0 days 00:18:16
5 2025-05-01 11:35:43       2025-05-01 11:31:27 0 days 00:04:16
6 2025-05-03 11:46:41       2025-05-03 11:31:38 0 days 00:15:03
7 2025-05-01 17:17:37       2025-05-01 17:14:00 0 days 00:03:37
8 2025-05-02 16:32:06       2025-05-02 16:30:08 0 days 00:01:58
9 2025-05-01 13:53:48                       NaT             NaT
64893


In [71]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

0

In [72]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'leads QCed in more than 01 hr'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)



In [73]:
data['Merchant QC TAT slab'].value_counts()

Merchant QC TAT slab
Leads QCed in between 02 - 05 min       13789
Leads QCed in less than 02 min          12851
Leads QCed in between 05 - 10 min        6525
Leads QCed in between 15 - 30 min        5704
Leads QCed in between 10 - 15 min        3530
Leads QCed in between 30 min - 01 hr      366
leads QCed in more than 01 hr              98
Name: count, dtype: int64

In [74]:
len(data)

64893

In [75]:
len(data[data['solution_type_level_3']=='3P FSE'])

129

In [76]:
data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE         2191
ACCOUNT_UPGRADE    1275
3P FSE              129
DIY                   2
Name: count, dtype: int64

In [77]:
data = data[~((data['Entity_Type'].isin(['PROPRIETORSHIP', 'NON_PROPRIETORSHIP'])) & (data['Product_Context'] == 'EDC'))]
len(data)

62015

In [78]:
data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE         2191
ACCOUNT_UPGRADE    1275
3P FSE              129
Name: count, dtype: int64

In [79]:
data.to_csv(out_file, index=False)

In [80]:
sas

NameError: name 'sas' is not defined

In [ ]:


# Convert to datetime
data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'])
data['Final TimeStamp'] = pd.to_datetime(data['Final TimeStamp'])

# Define active hours (08:30 AM - 09:30 PM)
active_start_time = pd.to_timedelta('08:30:00')
active_end_time = pd.to_timedelta('21:30:00')

# Function to calculate active duration for each row
def calculate_active_duration(row):
    start = row['Last QC Pending Timestamp']
    end = row['Final TimeStamp']
    
    # Check if either start or end is NaT
    if pd.isna(start) or pd.isna(end):
        return "00:00:00"  # Return 0 hours if either is NaT
    
    # Initialize total active duration
    total_active_duration = pd.Timedelta(0)
    
    # Iterate over each date from start to end
    for single_date in pd.date_range(start=start.date(), end=end.date()):
        active_start = pd.Timestamp(single_date) + active_start_time
        active_end = pd.Timestamp(single_date) + active_end_time
        
        # Determine the effective active period for this day
        effective_start = max(start, active_start)
        effective_end = min(end, active_end)

        # If there's a valid active period, add its duration
        if effective_start < effective_end:
            total_active_duration += effective_end - effective_start

    # Convert total active duration to hours
    total_hours = total_active_duration.total_seconds() / 3600
    
    # Format the hours to HH:MM:SS
    hours = int(total_hours)
    minutes = int((total_hours - hours) * 60)
    seconds = int(((total_hours - hours) * 60 - minutes) * 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Apply the function to each row
data['TAT'] = data.apply(calculate_active_duration, axis=1)

# Display the result
print(data[['Last QC Pending Timestamp', 'Final TimeStamp', 'TAT']])
print(len(data))


In [ ]:
# data['TAT']=data['Final TimeStamp']-data['Last QC Pending Timestamp']
# data['TAT']=data['TAT'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else 'NaT')
print(data.columns)
print(len(data))

In [ ]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

In [ ]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

In [ ]:
data["TAT"]=pd.to_timedelta(data['TAT'])



print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))

In [ ]:
type(data['TAT'][0])

In [ ]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 16:
        return 'Leads QCed in less than 15 min'
    elif 16 <= total_minutes < 36:
        return 'Leads QCed in between 15-30 min'
    elif 36 <= total_minutes < 66:
        return 'Leads QCed in between 30 min -1 hr'
    elif 66 <= total_minutes < 126:
        return 'Leads QCed in between 1 - 2 hr'
    elif 126 <= total_minutes < 186:
        return 'Leads QCed in between 2-3 hr'
    elif 186 <= total_minutes < 246:
        return 'Leads QCed in between 3-4 hr'
    else:
        return 'leads QCed in more than 4 hrs'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)


    



In [ ]:
print(data['Merchant QC TAT slab'].value_counts())
print(len(data))

In [ ]:
data['TAT'].head(10)

In [ ]:

# Function to convert timedelta to HH:MM:SS format with cumulative hours
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)  # Convert total seconds to hours
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'  # Format as HH:MM:SS

# Apply the function to convert timedelta to HH:MM:SS
data['TAT'] = data['TAT'].apply(format_timedelta)

In [ ]:
data.loc[(data['Lead Status']!='Merchant Live') & (data['Lead Status']!='Documents Rejected (Pending at Sales Team)'),'TAT']=''

data['TAT'].head(10)

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['Working TimeSlab','New Off Time','New Last QC Pending Time','Node id'])
data.columns

In [ ]:
col=data.pop('TAT')

t=data.columns.get_loc('Merchant QC TAT slab')

data.insert(t+1,'TAT',col)

data.columns

In [ ]:
data['solution_type_level_3'].unique()
len(data)

In [ ]:
data_acup_kyc=data[data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]
data_mco=data[~data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]

In [ ]:
#raise Exception("Stop execution here")

In [ ]:
with pd.ExcelWriter("all_flow_data_21st Oct'24.xlsx") as writer:
    # Write the original dataframe to the first sheet
    data.to_excel(writer, sheet_name='All Data', index=False)
    
    # Write data filtered by Category 'A' and 'C' to the second sheet
    data_acup_kyc.to_excel(writer, sheet_name=' KYC & ACC UPGD', index=False)
    
    # Write data filtered by Category 'B' and 'D' to the third sheet
    data_mco.to_excel(writer, sheet_name='MCO', index=False)

In [ ]:
data.to_csv("all_flow_data_19th Oct'24.csv", index=False)
